# 1. Load JSL Preds Data (with Ground Truths)

In [1]:
import os
import pandas as pd

file_name = 'data_with_jsl_preds.pkl'
data_path = os.path.join('..', 'data', file_name )
print("Raw data path:", data_path)
data_df = pd.read_pickle(data_path)
data_df.head(1)

Raw data path: ../data/data_with_jsl_preds.pkl


,id,text,chunk,ner_label,label_count,jsl_prediction
0,50,"Unfortunately , I think itis unlikelythathis s...","['he', 'over 9 to 12 months', 'he', 'he']","[Gender, RelativeDate, Gender, Gender]","{'Gender': 3, 'RelativeDate': 1}","[{'entity_type': 'Gender', 'chunk': 'he', 'sta..."


# 2. Task Selection

**Prediction Tasks**
1. 'Organ_Component'
   <br> *Evaluated for* 
    - External_body_part_or_region
    - Internal_organ_or_component
2. 'Gender'
3. 'Procedure_Treatment'
    <br> *Evaluated for* 
    - Procedure
    - Treatment
4. 'Problem' (19 Entities merged under Problem Type)
5. 'Medicine' (Drug_BrandName, Drug_Ingredient, Strength, Form, Dosage )
<br> In this evaluation case partial matches are considered as full_match
6. 'Drug'
    <br> *Evaluated for* 
    - Drug (Drug_BrandName, Drug_Ingredient)
    - Drug_Related (Strength, Form, Dosage)
7. 'Test'
8. 'Test_TestResults' (Test, Test Results)
    <br> *Evaluated for* 
    - Test
    - Test_Result

In [2]:
from modules.ProcessRawData import ProcessData
from modules.ProcessPredData import corrected_json, get_list_of_entities
import json

# read tasks from file
with open("./sources/tasks_list_jsl.json", "r") as f:
    data = json.load(f)

# extract problem_entity_list and task_list
task_list = data["task_list"]
eval_options_dict = data["eval_options_dict"]

print("task_list:", task_list.keys())
print("eval_options_dict:",eval_options_dict)

task_list: dict_keys(['Organ_Component', 'Gender', 'Procedure_Treatment', 'Problem', 'Medicine', 'Drug', 'Test', 'Test_TestResult'])
eval_options_dict: {'External_body_part_or_region': {'selected_entity_prediction': ['External_body_part_or_region'], 'selected_entity_gt': ['External_body_part_or_region'], 'gt_type_dict': {'External_body_part_or_region': 'External_body_part_or_region'}}, 'Internal_organ_or_component': {'selected_entity_prediction': ['Internal_organ_or_component'], 'selected_entity_gt': ['Internal_organ_or_component'], 'gt_type_dict': {'Internal_organ_or_component': 'Internal_organ_or_component'}}, 'Gender': {'selected_entity_prediction': ['Gender'], 'selected_entity_gt': ['Gender'], 'gt_type_dict': {'Gender': 'Gender'}}, 'Procedure': {'selected_entity_prediction': ['Procedure'], 'selected_entity_gt': ['Procedure'], 'gt_type_dict': {'Procedure': 'Procedure'}}, 'Treatment': {'selected_entity_prediction': ['Treatment'], 'selected_entity_gt': ['Treatment'], 'gt_type_dict': {

## 2.1 Select Task to Get Predictions

In [22]:
import ipywidgets as widgets
from modules.entity_selection import create_dropdown

# select entity for evaluation
entity_options = list(task_list.keys())

# create_dropdown(entity_options)
def assign_entity(entity):
    global entity_under_test
    entity_under_test = entity
    print("Predictions will be generated for the entities:", entity_under_test)

# create the dropdown menu
dropdown = widgets.Dropdown(options=entity_options)

# create the button
button = widgets.Button(description="Approve")

# define the callback function for the button
def on_button_click(button):
    assign_entity(dropdown.value)

# register the callback function with the button
button.on_click(on_button_click)

# display the dropdown menu and the button
display(dropdown)
display(button)


Dropdown(options=('Organ_Component', 'Gender', 'Procedure_Treatment', 'Problem', 'Medicine', 'Drug', 'Test', '…

Button(description='Approve', style=ButtonStyle())

Predictions will be generated for the entities: Organ_Component


# 3. Prepare Data for Prediction

In [23]:
# filter gt data for prediction based on selected entity under test and entity counts
covered_gt_label_list = task_list[entity_under_test]['covered_gt_label_list']
print("covered_gt_label_list:", covered_gt_label_list)
count_filter = task_list[entity_under_test]['count_filter']

prediction_source = "jsl_ner"

# process gt data for prediction and evaluation
process_gt_data = ProcessData(
    covered_gt_label_list, 
    count_filter, 
    data_path,
    prediction_source
)

processed_gt_df = process_gt_data.get_final_df()

2023-04-03 02:06:58,930 - ProcessRawData - INFO - main df shape: (849, 7)
2023-04-03 02:06:58,942 - ProcessRawData - INFO - filtered df shape: (62, 8)
2023-04-03 02:06:59,028 - ProcessRawData - INFO - final df shape: (62, 9)


covered_gt_label_list: ['Internal_organ_or_component', 'External_body_part_or_region']
Processing main csv...


# 5. Evaluation

## 5.1 Selection of entity for evaluation

In [24]:
import ipywidgets as widgets

# selection of entity type for benchmark
entity_list_for_benchmark = data["entity_for_benchmark"][entity_under_test]
print("entity_list_for_benchmark:", entity_list_for_benchmark)

if len(entity_list_for_benchmark) == 1:
    entity_for_benchmark = entity_list_for_benchmark[0]
    print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)
else:
    def assign_entity(entity):
        global entity_for_benchmark
        entity_for_benchmark = entity
        print("Benchmarks will be evaluated for the entity:", entity_for_benchmark)
    
    # create the dropdown menu
    dropdown = widgets.Dropdown(options=entity_list_for_benchmark)

    # create the button
    button = widgets.Button(description="Approve")

    # define the callback function for the button
    def on_button_click(button):
        assign_entity(dropdown.value)

    # register the callback function with the button
    button.on_click(on_button_click)

    # display the dropdown menu and the button
    display(dropdown)
    display(button)


entity_list_for_benchmark: ['External_body_part_or_region', 'Internal_organ_or_component']


Dropdown(options=('External_body_part_or_region', 'Internal_organ_or_component'), value='External_body_part_or…

Button(description='Approve', style=ButtonStyle())

Benchmarks will be evaluated for the entity: External_body_part_or_region


## 5.2 Run Evaluation 

In [25]:
# No user interaction needed, just run
from modules import Evaluation
import datetime
now = datetime.datetime.now()

eval_file_to_save = entity_for_benchmark + "_" + prediction_source + f"_eval_{now.strftime('%m%d_%H%M')}"

selected_entity_prediction = eval_options_dict[entity_for_benchmark]["selected_entity_prediction"]
print("selected_entity_prediction:", selected_entity_prediction)
selected_entity_gt = eval_options_dict[entity_for_benchmark]["selected_entity_gt"]
gt_type_dict = eval_options_dict[entity_for_benchmark]["gt_type_dict"]


evaluator = Evaluation.Evaluate(
    file_path=None, 
    dataframe=processed_gt_df, 
    prediction_source=prediction_source
)

eval_results = evaluator.get_match_counts(
    selected_entity_prediction, 
    selected_entity_gt, 
    eval_file_to_save,
    gt_type_dict
)

# write result to JSON file
with open('eval_results/eval_result.json', 'a') as f:
    json.dump(eval_results, f)
    
eval_results

selected_entity_prediction: ['External_body_part_or_region']
Evaluation results saved as ./eval_results/External_body_part_or_region_jsl_ner_eval_0403_0207.xlsx


{'version': 'External_body_part_or_region_jsl_ner_eval_0403_0207',
 'selected_entity_prediction': ['External_body_part_or_region'],
 'selected_entity_gt': ['External_body_part_or_region'],
 'full_match': 104,
 'accuracy_full_match': 0.84,
 'partial_match': 2,
 'accuracy_partial_match': 0.85,
 'no_match': 18,
 'gt_count': 124,
 'fp_count': 16}